# SVM trained by SMO（from scratch）

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
datasets = [train_data, test_data]

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
to_drop = ['Name', 'Ticket', 'Cabin', 'Embarked', 'PassengerId']

In [6]:
for dataset in datasets:
    dataset.drop(to_drop,axis=1,inplace=True)
    dataset.replace(to_replace='male',value=1,inplace=True)
    dataset.replace(to_replace='female',value=0,inplace=True)
    dataset['Age'].fillna(value=dataset['Age'].mean(), inplace=True)
    dataset['Fare'].fillna(value=dataset['Fare'].mean(), inplace=True)

In [7]:
print(datasets[0].isnull().sum())
print("-"*50)
print(datasets[1].isnull().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
dtype: int64
--------------------------------------------------
Pclass    0
Sex       0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64


In [8]:
labels = train_data['Survived']
train_data.drop('Survived', axis=1, inplace=True)
train_data = (train_data - train_data.mean())/train_data.std()
test_data = (test_data - test_data.mean())/test_data.std()

In [9]:
labels.replace(to_replace=0,value=-1,inplace=True);

In [12]:
train = train_data.iloc[:650,:]
y = labels[:650]
m = train.shape[0]
n = train.shape[1]
b = 0
w = np.zeros((n,))
alphas = np.zeros((m,))

eps = 1e-3

def getK(kernel):
    K = np.zeros((m,m))
    for i in range(m):
        for j in range(m):
            K[i,j] = kernel(train_data.iloc[i,:],train_data.iloc[j,:])
    return K

In [ ]:
def takeStep(i1,i2,kernel):
    if i1 == i2: return 0
    
    alpha1 = alpha[i1]
    alpha2 = alpha[i2]
    y1 = y[i1]
    y2 = y[i2]
    x1 = train.iloc[i1,:]
    x2 = train.iloc[i2,:]
    E1 = svm(x1,w,b) - y1
    E2 = svm(x2,w,b) - y2
    s = y1*y2
    
    # compute L,H via equations (13) and (14)
    if s < 0:
        L = max(0,alpha2-alpha1)
        H = min(C,C+alpha2-alpha1)
    elif s > 0:
        L = max(0,alpha2+alpha1-C)
        H = min(C,alpha2+alpha2)
    else:
        return 0
    
    k11 = kernel(x1,x1)
    k12 = kernel(x1,x2)
    k22 = kernel(x2,x2)
    
    eta = k11 + k22 - 2*k12
    
    if eta > 0:
        a2 = alpha2 + y2*(E1-E2) / eta
        if a2 < L:
            a2 = L
        elif a2 > H:
            a2 = H
    else:
        alpha[i2] = L
        Lobj = obj(alpha)
        alpha[i2] = H
        Hobj = obj(alpha)
        
        if Lobj < Hobj - eps:
            a2 = L
        elif Lobj > Hobj + eps:
            a2 = H
        else:
            a2 = alpha2
    
    if np.abs(a2-alpha2) < eps*(a2+alpha2+eps):
        return 0
    
    a1 = alpha1 + s*(alpha2-a2)
    
    b1 = E1 + y1()
    
        
def svm(x):
    ret = 0
    for i in range(m):
        ret += y[i]*alpha[i]*kernel(train.iloc[i,:],x)
    ret -= b
    return ret


def obj(alpha):
    alphay = alpha * y
    return alphay.dot(K).dot(alphay) - np.sum(alpha)